# CASE STUDY - performance monitoring (solution)

You will be building your own workflow template in this tutorial.  You
already have a Dockerfile and a basic Flask application to build an
API.  Lets combine what you have learned about logging to build a
``workflow-template`` that can be used to deploy models in a way that
facilitates performance monitoring.

There are three main parts to this case study.

1. Write unit tests for a logger and a logging API endpoint
2. Add logging to your Docker container
3. Add an API endpoint for logging
4. Make sure all tests pass
5. Create model performance investigative tooling
6. Swap out the iris data for the AAVAIL churn data

You may want to eventually rename the directory because in this
case-study you will swap out the iris data for `aavail-target.csv`.
It reality you will eventually want a library of workflow templates to
work from and the naming convention you decide on can help with
organization.  This notebook should reside in that source directory
regardless of the name.  We suggest that you go through all of the
tasks **first** using the iris data **then** copy the template to a
new folder and make it work for the AAVAIL churn data.  Eventually you
will want a suite of workflow templates that you will be able to
select from.

In [4]:
import os
import sys
import csv
import requests
from collections import Counter
from datetime import date
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Getting started

The ``workflow-template.zip`` is a workflow template.  Unpack the
directory in a location where you would like the source code to exist.
Leaving out the ``static`` directory that contains CSS and JavaScript
to render a landing page, the important pieces are shown in the
following tree.

```
├── app.py
├── Dockerfile
├── model.py
├── README.rst
├── requirements.txt
├── run-tests.py
├── templates
│   ├── base.html
│   ├── dashboard.html
│   ├── index.html
│   └── running.html
└── unittests
    ├── ApiTests.py
    ├── __init__.py
    ├── ModelTests.py
```

If you plan on modifying the HTML website you will need to modify the
files in ``templates``.  The rest of the files you should be familiar
with at this point.

## TASK 1: Write units test for a logger and a logging API endpoint

1. Using `model.py` and `./unittests/ModelTests.py` as an example create `logger.py` and 
  `./unittests/LoggerTests.py`.
2. Modify the files so that there are at a minimum the following tests:

    * ensure predict log is automatically created
    * ensure train log is automaticall created
    * ensure that the train function in model archives last used training data
    * ensure that 'n' predictions result in 'n' predict log entries
    * ensure that predict gracefully handles NaNs
    
> IMPORTANT: when writing to a log file from a unit test you will want to ensure that you do not modify or delete existing 'production' logs.  You can test your function with the following code (although it is likely easier to work directly in a terminal).

In [2]:
!python ./unittests/LoggerTests.py

....
----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


## TASK 2: Add an API endpoint for logging

In addition to the `predict` and `train` endpoints, create a third
endpoint that returns logs.  Remember that there are `train` and
`predict` log files and that they are set up to create new files each
month.  You will need to ensure that your endpoint can accommodate
this and the best way to ensure this is to **first write the unit
tests** then write the code.

Flask has several functions to help with the sending of files. One example is [send_from_directory](https://flask.palletsprojects.com/en/1.1.x/api/#flask.send_from_directory).

In [7]:
!pwd

/Users/marcohassan/Desktop/Learning/AI_workflow_Coursera/workflow-template/case-study-soln


In [30]:
!python ./unittests/ApiTests.py

..{'y_pred': [1], 'y_proba': [[0.004100209127047534, 0.573445936067756, 0.4224538548051963]]}
.

.
----------------------------------------------------------------------
Ran 4 tests in 0.085s

OK


## TASK 3: Make sure all tests pass

You have been working on specific suites of unit tests.  It is a best
practice to double-check that all tests pass after making major
changes like the ones you have just completed.

> make sure you modify the `./unittests/__init__.py` so that the
  LoggerTest suite is also included when running all tests.

In [39]:
!python ./run-tests.py

ssss..../Users/marcohassan/Desktop/python_venv/coursera_capstone/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/marcohassan/Desktop/python_venv/coursera_capstone/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/marcohassan/Desktop/python_venv/coursera_capstone/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.23.1. This might lead to

     0    1
0  6.1  2.8
E
ERROR: test_03_predict (ModelTests.ModelTest)
test the predict function input
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/marcohassan/Desktop/Learning/AI_workflow_Coursera/workflow-template/case-study-soln/unittests/ModelTests.py", line 58, in test_03_predict
    result = model_predict(query_data,model,test=True)
  File "/Users/marcohassan/Desktop/Learning/AI_workflow_Coursera/workflow-template/case-study-soln/model.py", line 175, in model_predict
    y_pred = model.predict(query)
  File "/Users/marcohassan/Desktop/python_venv/coursera_capstone/lib/python3.7/site-packages/sklearn/utils/metaestimators.py", line 119, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)
  File "/Users/marcohassan/Desktop/python_venv/coursera_capstone/lib/python3.7/site-packages/sklearn/pipeline.py", line 407, in predict
    Xt = transform.transform(Xt)
  File "/Users/marcohassan/De

## TASK 4: Create model performance investigative tooling

There are a lot of convenience functions you could create here.
Create them directly in this notebook or create them as scripts that
you may call from this notebook.

Essentially you will need to create a tools that compare the most
recently used training data to the most recent predictions.  In
reality your tooling should also allow you to compare predictions from
say one model to another.  The predictions come from log files.

For this task let's focus on the comparison of predictions to
established data.

1. Use bootstrap samples from the original data to ping the `predict`
endpoint.  Also, add a couple of outliers like you did in a few course
back with outlier detection.

2. Pull the predictions from the log file and summarize the investigation visually 

In [15]:
## YOUR CODE HERE

from monitoring import get_latest_train_data, get_monitoring_tools

## load latest data
data = get_latest_train_data()
y = data['y']
X = data['X']

In [16]:
## generate some data
bs_samples = 60
subset_indices = np.random.choice(np.arange(X.shape[0]),
                                  bs_samples,replace=True).astype(int)
mask = np.in1d(np.arange(X.shape[0]),subset_indices)
X_bs=X[mask]
X_outliers = X[:5].copy()
X_outliers['age'] = [88,90,76,80,68]
X_outliers['num_streams'] = [111,100,80,90,150]
X_query = pd.concat([X_bs,X_outliers])

print(X_query.shape)

(63, 4)


In [22]:
## delete the log file so we are starting fresh
today = date.today() 
logfile = os.path.join("logs","predict-{}-{}.log".format(today.year, today.month)) 
print(logfile)
if os.path.exists(logfile):
    os.remove(logfile)

## ping the API
request_json = {'query':X_query.to_dict(),'type':'dict'}
port = 5000
r = requests.post('http://0.0.0.0:{}/predict'.format(port),json=request_json)

## response = literal_eval(r.text)

##
## print(list(sorted(Counter(response['y_pred']).items())))

logs/predict-2020-7.log


SyntaxError: invalid syntax (<unknown>, line 1)

In [8]:
pm_tools = get_monitoring_tools(X,y)

In [9]:
## read in the logged data
df = pd.read_csv(logfile)
df.drop(columns=["unique_id","y_proba"], inplace=True)
df.head()

,timestamp,y_pred,query,model_version,runtime
0,1.576179e+09,0.0,"['singapore', 48, 'aavail_basic', 18]",0.1,000:00:00
1,1.576179e+09,0.0,"['united_states', 22, 'aavail_basic', 5]",0.1,000:00:00
2,1.576179e+09,0.0,"['singapore', 41, 'aavail_basic', 19]",0.1,000:00:00
3,1.576179e+09,1.0,"['singapore', 24, 'aavail_premium', 7]",0.1,000:00:00
4,1.576179e+09,0.0,"['united_states', 20, 'aavail_premium', 20]",0.1,000:00:00


In [10]:
## reconstruct a data frame from the logged queries
queries = [literal_eval(q) for q in df['query'].values]
queries = pd.DataFrame(queries)
queries.columns = ['country', 'age', 'subscriber_type', 'num_streams']
print(queries.shape)
queries.head()

(64, 4)


,country,age,subscriber_type,num_streams
0,singapore,48,aavail_basic,18
1,united_states,22,aavail_basic,5
2,singapore,41,aavail_basic,19
3,singapore,24,aavail_premium,7
4,united_states,20,aavail_premium,20


In [ ]:
from scipy.stats import wasserstein_distance
X_target = pm_tools['preprocessor'].transform(queries)

outlier_test = pm_tools['clf_X'].predict(X_target)
outliers_X = 100 * (1.0 - (outlier_test[outlier_test==1].size / outlier_test.size))
wasserstein_X = wasserstein_distance(pm_tools['X_source'].flatten(),X_target.flatten()) 
wasserstein_y = wasserstein_distance(pm_tools['y_source'],df['y_pred'].values)

if outliers_X >= pm_tools['outlier_X']:
    print("OUTLIER TEST FAILED: {} >= {}".format(round(outliers_X,2),
                                                 pm_tools['outlier_X']))
else:
    print("OUTLIER TEST PASSED: {} < {}".format(round(outliers_X,2),
                                                pm_tools['outlier_X']))
    
if wasserstein_X >= pm_tools['wasserstein_X']:
    print("DISTRIBUTION X TEST FAILED: {} >= {}".format(round(wasserstein_X,2),
                                                        pm_tools['wasserstein_X']))
else:
    print("DISTRIBUTION X TEST PASSED: {} < {}".format(round(wasserstein_X),
                                                       pm_tools['wasserstein_X']))
    
if wasserstein_y >= pm_tools['wasserstein_y']:
    print("DISTRIBUTION y TEST FAILED: {} >= {}".format(round(wasserstein_y,2),
                                                        pm_tools['wasserstein_y']))
else:
    print("DISTRIBUTION y TEST PASSED: {} < {}".format(round(wasserstein_y),
                                                       pm_tools['wasserstein_y']))

fig = plt.figure(figsize=(10,6),dpi=400)
ax = fig.add_subplot(111)

x_range = np.arange(outlier_test.size)
labels = ['outlier','normal']
markerline, stemlines, baseline = ax.stem(x_range, outlier_test, '-.',
                                          use_line_collection=True)
plt.setp(baseline, 'color', 'r', 'linewidth', 2)
ax.set_title("outlier visualization")
ax.set_ylabel("outlier=-1, normal=1")
ax.set_xlabel("queries (in time order)");

### SOLUTION NOTE

The tests we choose to run are reasonable given the size of the data.
We are saving each query and with the reconstructed queries we can
test for both outliers and distributional changes in the data.  All of
this code would be better organized under `monitoring.py` in a
production environment, but we walked through the process here with
the hope that it provides some insight.  Be cautioned that the
bootstrap and disk read/write portions of this code will take much
longer with large data sets and some optimization will be required.
For example, you could pre-train and serialize the outlier model(s).

## TASK 5: Swap out the iris data for the AAVAIL churn data

We suggest that you copy the iris example folder to a another directory, then re-create the template to work with the AAVAIL data.  The exercise of changing the dataset is very much aligned with real-world practices since you will often be modifying workflow-templates to meet the needs of a particular business opportunity.

In [ ]:
!python run-tests.py

In [ ]:
## this solution uses the AAVAIL data